In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import json


/Applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
intents = json.load(open('/Users/markwoolnough/scrapy/chatbot/intents.json'))

In [3]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')


In [4]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [5]:
p = bow("card-reader", words)
print (p)
print (classes)
print (len(words), "unique stemmed words", words)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
['AccountInfo', 'Balance', 'DigitalBanking', 'GoingAbroad', 'Iban', 'LostStolen', 'NoCardReader', 'Overdraft', 'PaymentDuration', 'goodbye', 'greeting', 'hours', 'opentoday', 'thanks']
80 unique stemmed words ["'s", 'a', 'abroad', 'account', 'am', 'and', 'anyon', 'ar', 'avail', 'bal', 'bank', 'be', 'between', 'bic', 'block', 'bye', 'can', 'card', 'card-reader', 'cod', 'dam', 'day', 'debit', 'diff', 'digit', 'do', 'find', 'for', 'going', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'ib', 'in', 'is', 'lat', 'limit', 'long', 'lost', 'mak', 'my', "n't", 'numb', 'op', 'or', 'ord', 'out', 'overdraft', 'pay', 'quickest', 'reg', 'replac', 'report', 'sav', 'see', 'should', 'sort', 'stol', 'tak', 'thank', 'that', 'the', 'ther', 'to', 'today', 'transf', 'want', 'way', 'what', 'when', 'wher', 'wil', 'yo', 'you']


In [6]:
# load our saved model
model.load('./model.tflearn')


INFO:tensorflow:Restoring parameters from /Users/markwoolnough/model.tflearn


In [7]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [8]:
REQUEST = json.dumps({
    'path' : {},
    'args' : {}
})

In [16]:
# GET /response/:phrase
request = json.loads(REQUEST)
name = request['path'].get('phrase', 'Greetings')
#print(name)
response(name)

Hello, thanks for visiting


In [10]:
classify('How long will a payment or transfer take to arrive?')

[('PaymentDuration', 0.98769611)]

In [18]:
response('What hours are you open?')

To find out what time a branch is open you can use our branch locator.  Our branch locator makes it easy to track down the closest branches and cash machines. http://locator-rbs.co.uk/


In [12]:
classify("hours")

[('greeting', 0.54547399), ('hours', 0.25406736)]

In [13]:
response("payment duration")

Any time!
